# Introduction

This is a basic example of using TOAST interactively for LiteBIRD simulations.  You can install the two packages with:

In [ ]:
# ! pip install --pre toast
# ! pip install https://github.com/hpc4cmb/litebirdtask/archive/main.zip

This notebook uses additional packages that can be installed with pip or conda depending on the tool you are using to manage your environment:

In [ ]:
#! pip install wurlitzer ipywidgets plotly plotly-resampler

Now import our modules:

In [ ]:
# Built-in modules
import sys
import os
from datetime import datetime
import tempfile

# External modules
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import healpy as hp

# LiteBIRD and TOAST tools

import toast
import toast.ops
from toast import pixels_io_healpix as pio
import toast.widgets

import litebirdtask as lbt
from litebirdtask import vis as lbtv
from litebirdtask import ops


# Capture C++ output in the jupyter cells
%load_ext wurlitzer

# Display inline plots
%matplotlib inline

# Data Simulation

We start by creating a simulated LiteBIRD observing campaign with a scan strategy based on the instrument model and a few other parameters.  We then simulate detector timestream components to produce a realistic data set for analysis.

## Instrument Model

The rest of the notebook requires a local copy of the IMO.  This can be downloaded from the wiki (for example).  This file contains not only instrument parameters, but also default scanning parameters.

In [ ]:
imo_file = "/home/kisner/data/litebird/IMoV2-14June.json"

## Simulated Observing

Next we are going to run a LiteBIRD scanning simulation.  We start with an empty TOAST data container.  For this notebook we are not using MPI, but MPI is supported both in standalone workflow scripts as well as notebooks using the MPI backend to ipyparallel.

The `SimObserve` operator here selects detectors based on regular expressions matching the telescope, channel, and wafer names.  The length of a science observation is a free parameter, along with the number of observations to simulate (up to the mission length).  The gap length is obtained from the duty cycle in the IMO, along with all the scanning parameters.

In [ ]:
# Starting with empty Data container
data = toast.Data()

In [ ]:
# Get help about an operator
?ops.SimObserve

In [ ]:
# Simulate
sim_obs = ops.SimObserve(
    imo_file=imo_file,
    select_telescope="LFT",
    select_channel="L1-040",
    select_wafer=None,
    observation_time=60.0 * u.minute,
    num_observation=24,
    detset_key="pixel",
)
sim_obs.apply(data)

## Pointing Model

Our pointing model consists of 3 pieces.  The first is the detector pointing operator which translates boresight quaternions to the detector frame, with the Z-axis of the detector frame pointed at the detector line of sight and the X-axis of the detector frame aligned with the polarization sensitive direction.  The second piece is an operator which computes the pixel indices given a detector's pointing on the sky.  The final piece is an operator which computes the Stokes weights for each detector sample.  In this example we are using default operators in TOAST, but could implement other operators specific to LiteBIRD (for example a more detailed HWP response).  We use a low resolution pixelization for this example.

In [ ]:
det_pntg = toast.ops.PointingDetectorSimple()

det_pixels = toast.ops.PixelsHealpix(
    nest=True,
    nside=128,
    detector_pointing=det_pntg,
)

det_weights = toast.ops.StokesWeights(
    mode="IQU",
    detector_pointing=det_pntg,
    hwp_angle=sim_obs.hwp_angle,
)

## Default Noise Model

We will estimate the noise below, but we can also create a default noise model based purely on nominal detector values.

In [ ]:
default_model = toast.ops.DefaultNoiseModel()
default_model.apply(data)

## Simulated Timestream Components

We can simulate a variety of detector data components.

### Dipole

This will simulate the solar plus orbital dipole, but the orbit is not simulated yet within the `LitebirdSite` class, and so this will only include the motion of the Earth.

In [ ]:
sim_dipole = toast.ops.SimDipole(
    freq=40.0 * u.GHz,
    mode="total",
)
sim_dipole.apply(data)

### Fake Sky

In order to avoid downloading a signal map in this notebook, we will just generate a fake synthetic sky.  This is just for visualization and has no physical meaning.

In [ ]:
lmax = 2 * det_pixels.nside
cl = np.zeros(4 * (lmax + 1)).reshape([4, -1])
cl[0, :] = 1.0e-9 * np.ones(lmax + 1)
cl[1, :] = 1.0e-10 * np.ones(lmax + 1)
cl[2, :] = 1.0e-11 * np.ones(lmax + 1)
fake_I, fake_Q, fake_U = hp.synfast(
    cl,
    det_pixels.nside,
    pol=True,
    lmax=lmax,
    fwhm=np.radians(3.0),
    verbose=False,
)

fake_sky_file = "fake_input_sky.fits"
hp.write_map(fake_sky_file, [fake_I, fake_Q, fake_U])

hp.mollview(fake_I, title="Fake Sky I", cmap="inferno")
hp.mollview(fake_Q, title="Fake Sky Q", cmap="inferno")
hp.mollview(fake_U, title="Fake Sky U", cmap="inferno")


In [ ]:
# Now scan this into a map
sim_map_scan = toast.ops.ScanHealpixMap(
    file=fake_sky_file,
    pixel_pointing=det_pixels,
    stokes_weights=det_weights,
)
sim_map_scan.apply(data)

### Instrumental Noise

For this small example, we are just simulating per-detector 1/f noise from the nominal noise model, but we could also simulate correlated noise by creating a suitable `Noise` object for each observation that had a mixing matrix describing the correlations.

In [ ]:
sim_noise = toast.ops.SimNoise(
    noise_model=default_model.noise_model,
)
sim_noise.apply(data)

### Saving Data

If we want to save this simulated data for later we can do that now.  Here we use the TOAST native HDF5 format with FLAC compression for the detector signal and gzip for the flags.

In [ ]:
hdf5_volume = "data"

save_hdf5 = toast.ops.SaveHDF5(
    volume=hdf5_volume,
    detdata=[
        (sim_obs.det_data, {"type": "flac"}),
        (sim_obs.det_flags, {"type": "gzip"}),
    ]
)

save_hdf5.apply(data)

# Data Exploration

You can directly access / modify / plot data stored within the TOAST containers.  Each `Observation` is independent, so for this exercise we can look at just the first observation.

In [ ]:
first_ob = data.obs[0]
print(first_ob)

In the last cell you can see that the `Observation` has several "shared" data fields containing the pointing information and some other empty types of data "detdata" and "intervals".  We can just print these like a numpy array:

In [ ]:
print(first_ob.shared["times"])

You can see that the "shared" data buffers are a special kind of array that (if MPI is being used) have only a single copy on each compute node.  You can access individual elements with normal slice notation, or you can get a numpy array view by accessing the `.data` attribute.  For example we can plot them:

In [ ]:
# Plot HWP angle vs time for the first observation
times = first_ob.shared["times"]
hwp = first_ob.shared["hwp_angle"]

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(times.data[:100], hwp.data[:100])

plt.show()

The `detdata` attribute of an observation contains just the local data on each process, so you can read and write to these arrays.  The named keys in this `detdata` dictionary allow us to access the data by detector name, detector index, or sample range:

In [ ]:
signal = first_ob.detdata["signal"]

In [ ]:
print(signal["000_000_003_QA_040_B"])

In [ ]:
print(signal[["000_000_003_QA_040_B", "000_000_003_QA_040_T"], 0:4])

In [ ]:
# The whole thing...
print(signal[:, :])

You can plot some detector data relative to the timestamps, for example:

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(
    times.data, 
    signal["000_000_003_QA_040_B"],
)

ax.plot(
    times.data, 
    signal["000_000_003_QA_040_T"],
)

plt.show()

The `Observation` class also gives us access to the focalplane properties for this observation:

In [ ]:
# The telescope for this observation
print(first_ob.telescope)

In [ ]:
# The focalplane
print(first_ob.telescope.focalplane)

In [ ]:
# The Table of detector properties
print(first_ob.telescope.focalplane.detector_data.info)
print(first_ob.telescope.focalplane.detector_data)

### Interactive Visualization

The next line launches an interactive ipython widget.  If you are running the whole notebook at once, comment out this line, since it will cause the kernel to run forever.

In [ ]:
# w = toast.widgets.ObservationWidget(data.obs[0])

# Data Reduction

Now we consider the previous data set and analyze this.  Here we use the data already simulated in memory, but could also load the data from disk.  For this example, we do not perform focalplane reconstruction- that is a future exercise.  We also do not demonstrate either HWP demodulation (which is supported in TOAST) or regression of HWP synchronous signal in the map-making (which is nearing completion).

## Noise Estimation

Here we estimate the noise assuming that the timestreams are noise dominated and fit those estimates to a 1/f model.  In practice, since we have not removed the dipole, we expect this to show up as a large additional 1/f model.

In [ ]:
noise_estim = toast.ops.NoiseEstim(
    out_model="noise_est",
    lagmax=2048,
    nbin_psd=64,
    nsum=1,
    naverage=128,
)
noise_estim.apply(data)

In [ ]:
noise_fit = toast.ops.FitNoiseModel(
    noise_model=noise_estim.out_model,
    out_model="fit_noise",
    least_squares_ftol=None,
    least_squares_xtol=1.0e-12,
    least_squares_gtol=None,
)
noise_fit.apply(data)

Plot the noise estimate and fit for a couple detectors within one observation.  Note the large 1/f due to the un-removed dipole.

In [ ]:
sim_noise = first_ob[default_model.noise_model]
est_noise = first_ob[noise_estim.out_model]
fit_noise = first_ob[noise_fit.out_model]

for idet in range(2):
    det_name = first_ob.local_detectors[idet]
    toast.vis.plot_noise_estim(
        None,
        est_noise.freq(det_name),
        est_noise.psd(det_name),
        fit_freq=fit_noise.freq(det_name),
        fit_psd=fit_noise.psd(det_name),
        true_net=sim_noise.NET(det_name),
        true_freq=sim_noise.freq(det_name),
        true_psd=sim_noise.psd(det_name),
        semilog=False,
    )

## Map Making

Next we will make a basic map using a single regression template representing baseline offsets.

In [ ]:
# Use a single Offset template (destriping baselines)
baselines = toast.templates.Offset(
    noise_model=noise_fit.out_model,
    step_time=1.0 * u.second,
)

In [ ]:
# Template matrix with these templates
tmatrix = toast.ops.TemplateMatrix(
    templates=[baselines],
)

In [ ]:
# Binning operator
binner = toast.ops.BinMap(
    pixel_pointing=det_pixels,
    stokes_weights=det_weights,
    noise_model=noise_fit.out_model,
)

In [ ]:
# Optionally save full detector pointing.  This speeds up the mapmaking dramatically,
# but results in a 5x increase in memory.
binner.full_pointing = True

In [ ]:
out_dir = "maps"
if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

mapper = toast.ops.MapMaker(
    binning=binner,
    template_matrix=tmatrix,
    solve_rcond_threshold=1e-3,
    map_rcond_threshold=1e-3,
    iter_min=40,
    iter_max=100,
    write_hits=True,
    write_map=True,
    write_binmap=True,
    write_cov=True,
    output_dir=out_dir,
)

In [ ]:
mapper.apply(data)

Now plot the output maps and residual compared to the input fake sky:

In [ ]:
mapfile_root = os.path.join(mapper.output_dir, mapper.name)

hits = hp.read_map(
    f"{mapfile_root}_hits.fits", 
    dtype=np.int32
)
hp.mollview(hits, title="Solved Map Hits", cmap="inferno", min=0, max=1000)

Imap, Qmap, Umap = hp.read_map(
    f"{mapfile_root}_map.fits", 
    field=None
)

hit_pix = hits > 0
unhit_pix = np.logical_not(hit_pix)
Imap[unhit_pix] = hp.UNSEEN
Qmap[unhit_pix] = hp.UNSEEN
Umap[unhit_pix] = hp.UNSEEN

I_range = 0.005
P_range = 0.0002
hp.mollview(Imap, title="Solved Map Stokes I", cmap="inferno", min=-I_range, max=I_range)
hp.mollview(Qmap, title="Solved Map Stokes Q", cmap="inferno", min=-P_range, max=P_range)
hp.mollview(Umap, title="Solved Map Stokes U", cmap="inferno", min=-P_range, max=P_range)

In [ ]:
# Plot the residual
Idiff = Imap - fake_I
Qdiff = Qmap - fake_Q
Udiff = Umap - fake_U
Idiff[unhit_pix] = hp.UNSEEN
Qdiff[unhit_pix] = hp.UNSEEN
Udiff[unhit_pix] = hp.UNSEEN

hp.mollview(Idiff, title="Solved Minus Input Stokes I", cmap="inferno", min=-I_range, max=I_range)
hp.mollview(Qdiff, title="Solved Minus Input Stokes Q", cmap="inferno", min=-P_range, max=P_range)
hp.mollview(Udiff, title="Solved Minus Input Stokes U", cmap="inferno", min=-P_range, max=P_range)